## CSE-4076 Lab Assignment 6
Devadharsanan 21MIA1163

## TASK 1

In [ ]:
import cv2

# Path to video file
video_path = r"C:\Users\devadharsanan\Videos\person_tracking.mp4"
cap = cv2.VideoCapture(video_path)

# Check if the video was loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    print("Video loaded successfully.")

# Background subtractor for motion detection
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Loop through frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for faster processing
    frame = cv2.resize(frame, (640, 480))

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)
    _, fg_mask = cv2.threshold(fg_mask, 250, 255, cv2.THRESH_BINARY)

    # Find contours for motion detection
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, "Person", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display frame with annotations
    cv2.imshow("Person Tracking", frame)

    # Update frame count and progress
    frame_count += 1
    print(f"Processing frame {frame_count}", end="\r")

    # Exit condition
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")


## TASK 2


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path to video file
video_path = r"C:\Users\devadharsanan\Videos\shopping_area.mp4"
cap = cv2.VideoCapture(video_path)

# Verify video load success
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    print("Video loaded successfully.")

# Video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frames_per_interval = fps * 120  # 2-minute intervals
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Track people count over intervals
people_count_per_interval = []
frame_count = 0
current_interval_count = 0

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame and apply background subtraction
    frame = cv2.resize(frame, (640, 480))
    fg_mask = bg_subtractor.apply(frame)
    _, fg_mask = cv2.threshold(fg_mask, 250, 255, cv2.THRESH_BINARY)

    # Find contours to detect people
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    people_count = sum(1 for c in contours if cv2.contourArea(c) > 500)

    # Accumulate people count for current interval
    current_interval_count += people_count
    frame_count += 1

    # If end of interval reached, save count and reset for next interval
    if frame_count % frames_per_interval == 0:
        people_count_per_interval.append(current_interval_count)
        current_interval_count = 0
        print(f"Interval {len(people_count_per_interval)}: {people_count_per_interval[-1]} people")

cap.release()

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(people_count_per_interval, marker='o')
plt.title("People Count Over Time Intervals")
plt.xlabel("Time Interval")
plt.ylabel("People Count")
plt.grid()
plt.show()


## TASK 3


In [ ]:
import cv2
import numpy as np

# Paths to reference image and video
reference_image_path = r"C:\Users\devadharsanan\Pictures\reference_face.jpg"
video_path = r"C:\Users\devadharsanan\Videos\faces_video.mp4"

# Load face cascade and reference image
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
ref_img = cv2.imread(reference_image_path, 0)

# Detect face in the reference image
faces = face_cascade.detectMultiScale(ref_img, scaleFactor=1.1, minNeighbors=5)
if len(faces) > 0:
    (x, y, w, h) = faces[0]
    ref_face = ref_img[y:y + h, x:x + w]
else:
    print("Error: No face detected in the reference image.")
    exit()

# Load the video
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale and detect faces
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    # Match each detected face to reference face
    for (x, y, w, h) in faces:
        detected_face = gray_frame[y:y + h, x:x + w]
        match = cv2.matchTemplate(detected_face, ref_face, cv2.TM_CCOEFF_NORMED)
        if match >= 0.6:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, "Suspect", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with matched faces
    cv2.imshow("Suspect Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## TASK 4

In [ ]:
import cv2

video_path = r"C:\Users\devadharsanan\Videos\entry_exit.mp4"
cap = cv2.VideoCapture(video_path)
bg_subtractor = cv2.createBackgroundSubtractorMOG2()
entries, exits = 0, 0
entry_y, exit_y = 200, 100

# Ensure video loads
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fg_mask = bg_subtractor.apply(frame)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Track and count entries/exits
    for c in contours:
        if cv2.contourArea(c) > 1000:
            x, y, w, h = cv2.boundingRect(c)
            center_y = y + h // 2
            if center_y > entry_y:
                entries += 1
            elif center_y < exit_y:
                exits += 1

    # Display counts on frame
    cv2.putText(frame, f"Entries: {entries}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(frame, f"Exits: {exits}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.imshow("Entry/Exit Counter", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## TASK 5

In [ ]:
import cv2
import time
import pandas as pd
import os

video_path = r"C:\Users\devadharsanan\Videos\mall_dwell_time.mp4"
output_folder = r"C:\Users\devadharsanan\Documents\dwelling_time_results"
os.makedirs(output_folder, exist_ok=True)
cap = cv2.VideoCapture(video_path)
bg_subtractor = cv2.createBackgroundSubtractorMOG2()
entry_times, dwell_times = {}, {}
tracked_positions = {}
entity_id_counter = 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fg_mask = bg_subtractor.apply(frame)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 1000:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        entity_id = None
        for eid, (ex, ey, ew, eh) in tracked_positions.items():
            if abs(x - ex) < 50 and abs(y - ey) < 50:
                entity_id = eid
                break
        if entity_id is None:
            entity_id = entity_id_counter
            entry_times[entity_id] = time.time()
            tracked_positions[entity_id] = (x, y, w, h)
            entity_id_counter += 1
        dwell_times[entity_id] = time.time() - entry_times[entity_id]
        cv2.putText(frame, f"Time: {dwell_times[entity_id]:.1f}s", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow("Dwelling Time Tracker", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save dwelling times to a CSV
pd.DataFrame([{'Entity ID': eid, 'Dwelling Time': t} for eid, t in dwell_times.items()]).to_csv(os.path.join(output_folder, 'dwelling_times.csv'), index=False)


## TASK 6

In [ ]:
import cv2
import os

# Path to the video
video_path = r"C:\Users\devadharsanan\Downloads\in.mp4"
output_folder = 'frames'

# Create a folder to save frames if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    frame_count = 0
    
    # Loop through each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()
        
        # If a frame is returned
        if ret:
            # Define filename for each frame
            frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
            
            # Save the frame as an image file
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
        else:
            break

    # Release the video capture object
    cap.release()
    print(f"Total frames saved: {frame_count}")


In [ ]:
import cv2
import os

# Paths to input and output folders
input_folder = 'frames'
output_folder = 'detected_cars_frames'
os.makedirs(output_folder, exist_ok=True)

# Initialize background subtractor for motion detection
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Loop through the saved frames
for frame_file in sorted(os.listdir(input_folder)):
    frame_path = os.path.join(input_folder, frame_file)
    frame = cv2.imread(frame_path)

    # Check if frame is loaded successfully
    if frame is None:
        continue

    # Apply background subtraction
    fg_mask = background_subtractor.apply(frame)
    
    # Threshold the mask to binary to get clean detection areas
    _, fg_mask = cv2.threshold(fg_mask, 254, 255, cv2.THRESH_BINARY)

    # Find contours to detect moving cars
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected = False
    for cnt in contours:
        # Filter out small areas to ignore noise
        if cv2.contourArea(cnt) > 500:  # Adjust area threshold based on car size
            x, y, w, h = cv2.boundingRect(cnt)
            # Draw bounding box on the frame (optional for visualization)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            detected = True

    # If any cars were detected in the frame, save it
    if detected:
        output_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_path, frame)

print("Car detection complete. Frames with detected cars are saved in:", output_folder)


In [ ]:
import cv2
import os
import numpy as np

# Paths to input and output folders
input_folder = 'frames'
output_folder = 'categorized_cars'
os.makedirs(output_folder, exist_ok=True)

# Initialize background subtractor
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Define color ranges for various colors
color_ranges = {
    "red": ((0, 100, 100), (10, 255, 255)),
    "orange": ((10, 100, 100), (25, 255, 255)),
    "yellow": ((25, 100, 100), (35, 255, 255)),
    "green": ((35, 50, 50), (85, 255, 255)),
    "cyan": ((85, 100, 100), (95, 255, 255)),
    "blue": ((95, 150, 0), (125, 255, 255)),
    "purple": ((125, 100, 100), (150, 255, 255)),
    "pink": ((150, 100, 100), (170, 255, 255)),
    "white": ((0, 0, 200), (180, 30, 255)),  # Bright area with low saturation
    "gray": ((0, 0, 50), (180, 20, 200)),    # Low saturation, moderate brightness
    "black": ((0, 0, 0), (180, 255, 50)),    # Very low brightness
}

# Process each saved frame
car_count = 0
for frame_file in sorted(os.listdir(input_folder)):
    frame_path = os.path.join(input_folder, frame_file)
    frame = cv2.imread(frame_path)

    if frame is None:
        continue

    # Apply background subtraction to detect moving objects
    fg_mask = background_subtractor.apply(frame)
    _, fg_mask = cv2.threshold(fg_mask, 254, 255, cv2.THRESH_BINARY)

    # Find contours of moving objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        # Filter out small contours to remove noise
        if cv2.contourArea(cnt) > 500:  # Adjust based on car size
            x, y, w, h = cv2.boundingRect(cnt)
            car_region = frame[y:y + h, x:x + w]

            # Convert to HSV for color-based classification
            hsv = cv2.cvtColor(car_region, cv2.COLOR_BGR2HSV)

            # Determine car color
            car_color = "unknown"
            for color_name, (lower, upper) in color_ranges.items():
                mask = cv2.inRange(hsv, lower, upper)
                if cv2.countNonZero(mask) > 500:  # Adjust threshold as needed
                    car_color = color_name
                    break

            # Save categorized frame with color label
            car_count += 1
            output_path = os.path.join(output_folder, f'{car_color}_car_{car_count:04d}.jpg')
            cv2.imwrite(output_path, car_region)

print("Car detection and categorization complete. Categorized frames are saved in:", output_folder)


In [ ]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Paths to input images and output folder for unique cars
input_folder = 'categorized_cars'
unique_folder = 'unique_cars'
os.makedirs(unique_folder, exist_ok=True)

# Set minimum dimensions for an image to be considered
min_width, min_height = 100, 100  # Adjust based on your images

# Threshold for SSIM similarity to consider two images as duplicates
similarity_threshold = 0.8

# Set a common size for all images for SSIM comparison
fixed_size = (200, 200)

# Function to check similarity between two images using SSIM
def are_images_similar(img1, img2, threshold=similarity_threshold):
    # Resize both images to the fixed size
    img1_resized = cv2.resize(img1, fixed_size)
    img2_resized = cv2.resize(img2, fixed_size)
    
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1_resized, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2_resized, cv2.COLOR_BGR2GRAY)
    
    # Compute SSIM between the resized grayscale images
    s = ssim(img1_gray, img2_gray)
    return s > threshold

# List to store unique images
unique_images = []
car_count = 0

# Process each image in the input folder
for image_file in sorted(os.listdir(input_folder)):
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)

    # Skip images that are too small
    if image.shape[1] < min_width or image.shape[0] < min_height:
        continue

    # Compare with existing unique images
    is_unique = True
    for unique_image in unique_images:
        if are_images_similar(image, unique_image):
            is_unique = False
            break

    # If image is unique, add it to the unique images list and save it
    if is_unique:
        unique_images.append(image)
        car_count += 1
        output_path = os.path.join(unique_folder, f'unique_car_{car_count:04d}.jpg')
        cv2.imwrite(output_path, image)

print(f"Unique car detection complete. Total unique cars counted: {car_count}")
print("Unique car images are saved in:", unique_folder)


In [ ]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.cluster import KMeans

# Paths to the input and output folders
input_folder = 'unique_cars'
distinct_folder = 'distinct_cars'
os.makedirs(distinct_folder, exist_ok=True)

# Threshold for SSIM similarity to consider two images as duplicates
similarity_threshold = 0.4

# Minimum dimensions for an image to be considered distinct
min_width, min_height = 200, 150

# Set a common size for all images for SSIM comparison
fixed_size = (200, 200)

# Define a threshold for color similarity (Euclidean distance in RGB space)
color_similarity_threshold = 50

# Function to get the dominant color of an image
def get_dominant_color(image, k=1):
    # Resize image to speed up processing
    img = cv2.resize(image, (100, 100))
    img = img.reshape((-1, 3))

    # Apply k-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(img)
    dominant_color = kmeans.cluster_centers_[0]
    return dominant_color

# Function to check similarity between two images using SSIM
def are_images_similar(img1, img2, threshold=similarity_threshold):
    # Resize both images to the fixed size
    img1_resized = cv2.resize(img1, fixed_size)
    img2_resized = cv2.resize(img2, fixed_size)
    
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1_resized, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2_resized, cv2.COLOR_BGR2GRAY)
    
    # Compute SSIM between the resized grayscale images
    s = ssim(img1_gray, img2_gray)
    return s > threshold

# Function to calculate the Euclidean distance between two RGB colors
def color_distance(color1, color2):
    return np.linalg.norm(color1 - color2)

# List to store unique images and their dominant colors
distinct_images = []
distinct_colors = []
distinct_count = 0

# Process each image in the unique_cars folder
for image_file in sorted(os.listdir(input_folder)):
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)

    # Skip images that are smaller than the specified dimensions
    if image.shape[1] < min_width or image.shape[0] < min_height:
        continue

    # Calculate the dominant color of the current image
    dominant_color = get_dominant_color(image)

    # Compare with existing distinct images
    is_distinct = True
    for i, distinct_image in enumerate(distinct_images):
        # Check if images are similar based on SSIM
        if are_images_similar(image, distinct_image):
            # Check if dominant colors are also similar
            if color_distance(dominant_color, distinct_colors[i]) < color_similarity_threshold:
                is_distinct = False
                break

    # If image is distinct, add it to the distinct images list and save it
    if is_distinct:
        distinct_images.append(image)
        distinct_colors.append(dominant_color)
        distinct_count += 1
        output_path = os.path.join(distinct_folder, f'distinct_car_{distinct_count:04d}.jpg')
        cv2.imwrite(output_path, image)

print(f"Filtering complete. Total distinct cars saved: {distinct_count}")
print("Distinct car images are saved in:", distinct_folder)


In [ ]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans

# Paths to input folder and output folder for unique color cars
input_folder = 'distinct_cars'
output_folder = 'unique_color_cars'
os.makedirs(output_folder, exist_ok=True)

# Define a threshold for color similarity (Euclidean distance in RGB space)
color_similarity_threshold = 50

# Function to get the dominant color of an image
def get_dominant_color(image, k=1):
    # Resize image to speed up processing
    img = cv2.resize(image, (100, 100))
    img = img.reshape((-1, 3))

    # Apply k-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(img)
    dominant_color = kmeans.cluster_centers_[0]
    return dominant_color

# Function to calculate the Euclidean distance between two RGB colors
def color_distance(color1, color2):
    return np.linalg.norm(color1 - color2)

# List to store unique colors and their corresponding images
unique_colors = []
car_count = 0

# Process each image in the distinct_cars folder
for image_file in sorted(os.listdir(input_folder)):
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)

    # Calculate the dominant color of the current image
    dominant_color = get_dominant_color(image)

    # Check if this dominant color is similar to any previously saved unique color
    is_unique_color = True
    for color in unique_colors:
        if color_distance(dominant_color, color) < color_similarity_threshold:
            is_unique_color = False
            break

    # If the color is unique, save the image and add the color to the list
    if is_unique_color:
        unique_colors.append(dominant_color)
        car_count += 1
        output_path = os.path.join(output_folder, f'unique_color_car_{car_count:04d}.jpg')
        cv2.imwrite(output_path, image)

print(f"Color-based filtering complete. Total unique color cars saved: {car_count}")
print("Unique color car images are saved in:", output_folder)
